In [ ]:
%load_ext dotenv
%dotenv secrets.env

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI, AzureChatOpenAI

# Choose either the OpenAI API
llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
    model_kwargs={"seed": 42},
)

# ... or the Azure OpenAI API
# llm = AzureChatOpenAI(
#     model="gpt-4-turbo-128k-1106",
#     temperature=0,
#     model_kwargs={"seed": 42},
# )

# Calling the LLM directly


In [ ]:
llm.invoke("How are you?")

# Using an output parser in a LCEL chain

Read about the Runnable interface in the [LCEL documentation](https://python.langchain.com/docs/expression_language/).


In [ ]:
chain = llm | StrOutputParser()

chain.invoke("How are you?")

# Using prompt templates


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an helpful assistant."),
        ("human", "Where was the Olympics held in 1994?"),
        ("ai", "The 1994 Winter Olympics were held in Lillehammer, Norway."),
        ("human", "{question}"),
        ("system", "Answer in {language}."),
    ]
)

chain = template | llm | StrOutputParser()

chain.invoke(
    {
        "question": "Who won the 50k?",
        "language": "Norwegian",
    }
)

# Revisiting the similarity measure


In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are measuring semantic text similarity."
                "You report a score between 0 and 1."
            ),
        ),
        (
            "human",
            (
                "Below are some sentences. Compare them pairwise."
                "\n\n{formatted_sentences}"
            ),
        ),
    ]
)


def format_sentences(sentences: list[str]) -> str:
    return "\n".join(sentences)


chain = (
    {"formatted_sentences": RunnableLambda(format_sentences)}
    | template
    | llm
    | StrOutputParser()
)

print(
    chain.invoke(
        [
            "That sounds like a good idea!",
            "That does not sound like a good idea!",
            "I like your proposal!",
        ]
    )
)

# Producing structured output


In [ ]:
from pydantic.v1 import BaseModel, Field


class TextSimilarityResponse(BaseModel):
    sentence_A: str = Field(description="Sentence A.")
    sentence_B: str = Field(description="Sentence B.")
    reason: str = Field(
        description="The reason for the similarity score between A and B."
    )
    similarity_score: float = Field(description="The similarity score.")


class TextSimilarityResponses(BaseModel):
    responses: list[TextSimilarityResponse] = Field(
        description="The similarity responses."
    )

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.chains.openai_functions import get_openai_output_parser

functions = [convert_to_openai_function(TextSimilarityResponses)]
output_parser = get_openai_output_parser([TextSimilarityResponses])

chain = (
    {"formatted_sentences": RunnableLambda(format_sentences)}
    | template
    | llm.bind(functions=functions)
    | output_parser
)

In [ ]:
output = chain.invoke(
    [
        "That sounds like a good idea!",
        "That does not sound like a good idea!",
        "I like your proposal!",
    ]
)

In [ ]:
print(output.json(indent=2))